In [2]:
import numpy as np
import cv2
import sys
import os.path
import matplotlib.pyplot as plt
# matplotlib inline


In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15.0,15.0)
matplotlib.rcParams['image.cmap'] = 'gray'